<a href="https://colab.research.google.com/github/gen-ai-capstone-project-bartender-agent/MOK-5-ha/blob/main/notebooks/gradio_ui_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing for Kaggle Submission

This notebook is primarily for testing Gradio UI in-notebook. Note that this is not the only valid way in which we can test our use of Gradio, but rather that once we acheieve a desired result in an IDE, we must ensure it can be implemented here as well.

In [1]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy thinc spacy fastai google-cloud-bigquery
!pip install "google-generativeai>=0.3.0" "langgraph>=0.0.10" "requests>=2.31.0" "websockets>=12.0" "tenacity>=8.2.3" "gradio>=4.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 16.3 MB/s eta 0:00:00


Simply add your Gemini API key to the 'key' icon on the left sidebar and you can run the notebook.

In [5]:
# --- Imports ---
import logging
import os
import sys
from typing import Dict, List, Optional, Tuple

# UI / Display
import gradio as gr

# Gemini - Frontier LLM
try:
    import google.generativeai as ggenai
    from google.api_core import retry as core_retry
    from google.generativeai import types as genai_types
    #from google.generativeai import errors as genai_errors
except ImportError:
    print("Error: google.generativeai library not found.")
    print("Please install it using: pip install google-generativeai")
    sys.exit(1)

# Tenacity for retries
from tenacity import (
    retry as tenacity_retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type,
    before_sleep_log
)

# Attempt to import userdata for Colab, fallback to environment variables
try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False
    # from dotenv import load_dotenv # Uncomment if using python-dotenv locally
    # load_dotenv()

# --- Configuration ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Get API Key
GOOGLE_API_KEY = None
if IS_COLAB:
    try:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
        logger.info("Retrieved GOOGLE_API_KEY from Colab userdata.")
    except Exception as e:
        logger.warning(f"Could not get GOOGLE_API_KEY from Colab userdata: {e}")
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if GOOGLE_API_KEY:
        logger.info("Retrieved GOOGLE_API_KEY from environment variable.")
if not GOOGLE_API_KEY:
    logger.error("FATAL: GOOGLE_API_KEY not found in Colab userdata or environment variables.")
    raise EnvironmentError("GOOGLE_API_KEY is required but not found.")

# Configure Gemini Client and Model (Initialized ONCE)
try:
    ggenai.configure(api_key=GOOGLE_API_KEY)
    MODEL_NAME = 'gemini-1.5-flash' # Verify this model name
    model = ggenai.GenerativeModel(MODEL_NAME)
    logger.info(f"Successfully initialized Gemini model: {MODEL_NAME}")
except Exception as e:
    logger.exception(f"Fatal: Failed to initialize Gemini model: {str(e)}")
    raise RuntimeError(f"Failed to initialize Gemini model. Check API key and model name ('{MODEL_NAME}').") from e

# --- Static Data ---
# Define the Menu (Doesn't change per session)
menu: Dict[str, Dict[str, float]] = {
    "1": {"name": "Old Fashioned", "price": 12.00},
    "2": {"name": "Margarita", "price": 10.00},
    # ... (rest of the menu items) ...
    "9": {"name": "Negroni", "price": 11.00},
    "10": {"name": "Cosmopolitan", "price": 12.00}
}

# --- Core Agent Logic (Now Stateless Functions) ---

def get_menu_text() -> str:
    """Generates the menu text (Stateless)."""
    # No change needed, doesn't access mutable state
    menu_text = "Menu:\n" + "-"*5 + "\n"
    for item_id, item in menu.items():
        menu_text += f"{item_id}. {item['name']} - ${item['price']:.2f}\n"
    return menu_text

# Define retryable exceptions for the API call
# RETRYABLE_EXCEPTIONS = (
    # genai_errors.ResourceExhaustedError,
    # genai_errors.InternalServerError,
    # genai_errors.ServiceUnavailableError,
#)

@tenacity_retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    #retry=retry_if_exception_type(RETRYABLE_EXCEPTIONS),
    before_sleep=before_sleep_log(logger, logging.WARNING),
    reraise=True
)
def _call_gemini_api(prompt_content: List[str], config: Dict) -> ggenai.types.GenerateContentResponse:
    """Internal function to call the Gemini API with retry logic (Stateless)."""
    logger.debug("Calling Gemini API...")
    # Uses the globally initialized 'model'
    response = model.generate_content(
        contents=prompt_content,
        generation_config=config,
    )
    logger.debug("Gemini API call successful.")
    return response

def process_order(
    user_input_text: str,
    current_session_history: List[Dict[str, str]],
    current_session_order: List[Dict[str, float]]
) -> Tuple[str, List[Dict[str, str]], List[Dict[str, float]]]:
    """
    Processes user input using Gemini, updates state for the CURRENT SESSION.
    Accepts session history and order, returns (response_text, updated_history, updated_order).
    """
    if not user_input_text:
        logger.warning("Received empty user input.")
        # Return current state unchanged with a message
        return "Please tell me what you'd like to order.", current_session_history, current_session_order

    # Local copies for modification within this function call
    updated_history = current_session_history[:]
    updated_order = current_session_order[:]

    try:
        # --- Construct the prompt using session-specific history/order ---
        prompt_context = [
            "You are a friendly and helpful bartender...", # Keep instructions
            "\nHere is the menu:",
            get_menu_text(),
            "\nCurrent order:",
        ]
        if updated_order: # Use the passed-in order
            order_text = "\n".join([f"- {item['name']} (${item['price']:.2f})" for item in updated_order])
            prompt_context.append(order_text)
        else:
            prompt_context.append("No items ordered yet.")

        prompt_context.append("\nConversation History (latest turns):")
        history_limit = 10
        limited_history_for_prompt = updated_history[-history_limit:] # Use passed-in history
        for entry in limited_history_for_prompt:
             role = entry.get("role", "unknown").capitalize()
             content = entry.get("content", "")
             prompt_context.append(f"{role}: {content}")

        prompt_context.append(f"\nUser: {user_input_text}")
        prompt_context.append("\nBartender:")

        full_prompt = "\n".join(prompt_context)
        logger.info(f"Processing user input for session: {user_input_text}")
        logger.debug(f"Full prompt for Gemini:\n------\n{full_prompt}\n------")

        # --- Call Gemini API ---
        config_dict = {'temperature': 0.7, 'max_output_tokens': 2048}
        response = _call_gemini_api(prompt_content=[full_prompt], config=config_dict)

        # --- Process Response ---
        agent_response_text = ""
        # (Error/Safety checking logic for 'response' remains largely the same as before)
        if not response.candidates:
             logger.error("Gemini response has no candidates.")
             # ... (handle blocked prompts, etc.) ...
             agent_response_text = "Sorry, I couldn't generate a response. Please try again."
        elif not response.candidates[0].content or not response.candidates[0].content.parts:
             logger.error("Gemini response candidate is empty.")
             # ... (handle finish reasons like SAFETY, MAX_TOKENS etc.) ...
             agent_response_text = "Sorry, I had trouble generating a complete response."
        else:
             agent_response_text = response.candidates[0].content.parts[0].text
             logger.info(f"Gemini response received: {agent_response_text}")

             # --- Update Order Based on Response (Heuristic) ---
             # Modifies the 'updated_order' local variable
             for item_id, item in menu.items():
                 item_name_lower = item["name"].lower()
                 response_lower = agent_response_text.lower()
                 if item_name_lower in response_lower and \
                    any(add_word in response_lower for add_word in ["added", "adding", "got it", "sure thing"]):
                      if not updated_order or item["name"] != updated_order[-1]["name"]:
                          updated_order.append(item) # Append to local copy
                          logger.info(f"Heuristic: Added '{item['name']}' to session order.")
                          break

        # --- Update Session History ---
        # Append user input and assistant response to the local history copy
        updated_history.append({'role': 'user', 'content': user_input_text})
        updated_history.append({'role': 'assistant', 'content': agent_response_text})

        # --- Return updated state for this session ---
        return agent_response_text, updated_history, updated_order

    except Exception as e:
        logger.exception(f"Critical error in process_order: {str(e)}")
        # Return current state unchanged and an error message
        error_message = "I'm sorry, an unexpected error occurred. Please try again later."
        # Append only the error message to history to inform the user
        updated_history.append({'role': 'user', 'content': user_input_text}) # Keep user msg
        updated_history.append({'role': 'assistant', 'content': error_message})
        return error_message, updated_history, updated_order

# Note: reset_order function is removed as state reset is handled by Gradio callbacks


# --- Gradio Interface Callbacks (Using Session State) ---

def handle_gradio_input(
    user_input: str,
    session_history_state: List[Dict[str, str]],
    session_order_state: List[Dict[str, float]]
) -> Tuple[str, List[Dict[str, str]], List[Dict[str, str]], List[Dict[str, float]]]:
    """
    Gradio callback: Takes user input and session state, calls the agent,
    and returns updated UI and session state.
    """
    logger.info(f"Gradio input: '{user_input}'")
    logger.debug(f"Received session history state (len {len(session_history_state)}): {session_history_state}")
    logger.debug(f"Received session order state (len {len(session_order_state)}): {session_order_state}")

    # Call the stateless processing function with the current session's state
    response_text, updated_history, updated_order = process_order(
        user_input,
        session_history_state,
        session_order_state
    )

    # Return values to update Gradio components:
    # 1. Textbox value (clear it)
    # 2. Chatbot value (the updated history)
    # 3. History state value (persist updated history for next turn)
    # 4. Order state value (persist updated order for next turn)
    return "", updated_history, updated_history, updated_order

def clear_chat_state() -> Tuple[List, List, List]:
    """Gradio callback: Clears UI and session state by returning initial values."""
    logger.info("Clear button clicked - Resetting session state.")
    # Return empty lists for Chatbot, history state, and order state
    return [], [], []


# --- Gradio UI Definition (with gr.State) ---

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Bartending Agent")
    gr.Markdown("Welcome! Each session is independent. Ask me for a drink or check your order.")

    # --- Define Session State Variables ---
    # These hold the history and order specific to each user's session
    history_state = gr.State([]) # Initialize state for conversation history
    order_state = gr.State([])   # Initialize state for current order

    with gr.Row():
        with gr.Column(scale=2):
            # Chatbot display - its value is directly controlled by handle_gradio_input's return
            chatbot_display = gr.Chatbot(
                [], # Start empty, value comes from handle_gradio_input
                elem_id="chatbot",
                label="Conversation",
                bubble_full_width=False,
                height=500,
                type="messages"
            )
            msg_input = gr.Textbox(
                label="Your Order / Message",
                placeholder="What can I get for you?"
            )
            with gr.Row():
                clear_btn = gr.Button("Clear Conversation")
                submit_btn = gr.Button("Send", variant="primary")

        with gr.Column(scale=1):
             gr.Markdown("### Menu")
             gr.Markdown(get_menu_text(), elem_id="menu-display")
             # Future enhancement: Could add a gr.Textbox or gr.JSON here
             # bound to order_state to display the current order live.

    # --- Event Handlers ---
    # Define inputs and outputs including the state variables
    submit_inputs = [msg_input, history_state, order_state]
    submit_outputs = [msg_input, chatbot_display, history_state, order_state]

    # Link Textbox submit (Enter key)
    msg_input.submit(handle_gradio_input, submit_inputs, submit_outputs)

    # Link Submit button click
    submit_btn.click(handle_gradio_input, submit_inputs, submit_outputs)

    # Link Clear button click
    clear_outputs = [chatbot_display, history_state, order_state]
    clear_btn.click(clear_chat_state, None, clear_outputs)


# --- Launch the Gradio Interface ---
if __name__ == "__main__":
    logger.info("Launching Gradio interface...")
    demo.launch(debug=True, share=True) # Set share=True if public link needed
    logger.info("Gradio interface closed.")


<ipython-input-5-abbb39028a80>:265: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot_display = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6355171aa12693a125.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6355171aa12693a125.gradio.live
